In [7]:
from newspaper import Article
url = 'https://finance.naver.com/news/news_read.nhn?article_id=0004330323&office_id=008&mode=mainnews&type=&date=2019-12-26&page=1'

In [8]:
a = Article(url, language='ko')

In [9]:
a.download()
a.parse()

In [10]:
print(a.title)

네이버 금융


In [11]:
print(a.text)

(워싱턴 AFP=뉴스1) 우동명 기자 = 도널드 트럼프 미국 대통령 © AFP=뉴스1 <저작권자 © 뉴스1코리아, 무단전재 및 재배포 금지> (워싱턴 AFP=뉴스1) 우동명 기자 = 도널드 트럼프 미국 대통령 © AFP=뉴스1

▶최현석, 레스토랑 '조용히' 떠난 이유

▶장모 명의로 바뀐 집, 이혼하면…

▶네이버 구독하기

[머니투데이 조준영 기자] [[개장전]"美 단기 과열신호 켜졌지만 강세장 종료 신호는 아냐"]크리스마스를 하루 앞둔 지난 24일(현지시간) 뉴욕증시는 혼조세로 마감했다. 도널드 트럼프 미국 대통령이 미중 1단계 무역합의를 연내 공식 서명하겠다는 방침을 밝히고 리커창 중국 총리도 은행 지급준비율(RRR·지준율) 인하를 시시하는 등 호재가 잇따랐지만 최근 주가급등에 따른 부담감이 시장을 주춤하게 한 것으로 보인다.이날 뉴욕증시에서 블루칩(우량주) 클럽인 다우존스산업평균지수는 전날보다 36.08포인트(0.13%) 떨어진 2만8515.45에 거래를 마쳤다. 대형주 위주의 S&P(스탠다드앤푸어스) 500 지수는 0.63포인트(0.02%) 내린 3223.38을 기록했다.반면 기술주 중심의 나스닥종합지수는 7.24포인트(0.08%) 오른 8952.88에 마감하며 9거래일 연속 사상최고치를 경신했다. 1998년 이후 20여년만에 최장기간 연속 신고가 기록이다.한편 리커창 총리는 이날 쓰촨성 청두에서 현지 은행을 시찰하던 중 "추가적으로 전면적 지준율 인하와 선별적 지준율 인하를 검토할 것"이라며 "이를 통해 중소기업들의 자금난을 가시적으로 완화하겠다"고 말했다.지준율은 은행들이 고객들로부터 받은 예금 가운데 중앙은행에 의무적으로 적립해야 하는 비율을 말한다. 지준율이 낮아지면 은행이 대출에 쓸 수 있는 자금이 늘어나면서 시중에 더 많은 돈이 풀리게 된다.미중간 우호적인 분위기가 이어지고 있지만 지나친 주가급등을 경계하는 목소리도 적잖다. 실적 대비 주가상승 속도가 가파르다는 설명이다.오태동 NH투자증권 투자전략부장은 "미국 주식시장의 위험 선호심리는 

In [12]:
from konlpy.tag import Kkma, Okt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

In [13]:
# 텍스트 크롤링 > 문장 단위 분리 > 명사 추출
class SentenceTokenizer:
    
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", 
                          "중앙일보", "조선일보", "기자" ,"아", "휴", "아이구", "아이쿠", "아이고", 
                          "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",]
    
    def url2sentences(self, url):
        article = Article(url, language='ko')
        article.download()
        article.parse()
        sentences = self.kkma.sentences(article.text)
        for idx in range(len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
                                       if noun not in self.stopwords and len(noun) > 1]))
        return nouns

In [14]:
class GraphMatrix:
    
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    
    def build_word_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}